# $Q$-learning on cartpole
Below, we run the $Q$-learning algorithm on the cartpole example. 

## Importing libraries
First, we import the required libraries. If you get an error, you have possibly forgotten to change the kernel. See [Prepare a virtual environment](Preparation.ipynb).

In [1]:
import numpy as np
import tensorflow as tf
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
from policy_iteration import PI
from dynamics import CartPole

## Saving directories
Next, we set up some paths to write data and capture some videos for future investigation.

In [2]:
STORE_PATH = '/tmp/cartpole_exp1/Q'
data_path = STORE_PATH + f"/data_{dt.datetime.now().strftime('%d%m%Y%H%M')}"
agent_path = STORE_PATH + f"/agent_{dt.datetime.now().strftime('%d%m%Y%H%M')}"
train_writer = tf.summary.create_file_writer(data_path)

## Making the environment
We select the random seed and make the cartpole environment.


In [3]:
Rand_Seed = 1
env_par = {
    'Rand_Seed': Rand_Seed,
    'STORE_PATH': STORE_PATH,
    'monitor': False,
    'threshold': 195.0
}
Rand_Seed = 1
CP = CartPole(env_par)


## Making the $Q$-learning agent
We make the $Q$-learning agent. You can change the following hyper parameters if you like.
* `hidden_size`: Number of nodes in the layers.
* `GAMMA`: forgetting factor in the total cost. It should be in $[0\:1]$.
* `num_episodes`: The maximum number of episodes to run.
* `epsilon`: The probability of exploration. It should be in $(0\:1)$.
* `learning_rate_adam`: The learning rate for adam optimization.
* `adam_eps`: The epsilon in adam optimization.

In [4]:
agent_par = {
    'num_state': CP.env.observation_space.shape[0],
    'num_actions': CP.env.action_space.n,
    'Rand_Seed': Rand_Seed,
    'hidden_size': 30,
    'GAMMA': 1.0,
    'num_episodes': 5000,
    'epsilon': 0.1, 
    'learning_rate_adam': 0.001,
    'adam_eps': 0.1,
}
policy = PI(agent_par)


## Start learning
Now, we start the learning loop. The learning loop itertes for a maximum of number of `num_episodes`. In each iteration
* The agent derives the environment for one episode. It is called a rollout.
* We update the agent by $Q$-learning using the recorded data.
* We check if the problem is solved.
* We write the data.
At the end of the learning loop, we close the environment.

In [5]:
tot_rews = []
mean_100ep = 0
for episode in range(agent_par['num_episodes']):

    # Do one rollout
    states, actions, rewards, new_states, dones = CP.one_rollout(policy)

    # Update the agent
    loss = policy.update_network(states, actions, rewards, new_states, dones)

    # Check if the problem is solved
    if episode > 100:
        mean_100ep = np.mean(tot_rews[-101:-1])

    tot_reward = sum(rewards)
    tot_rews.append(tot_reward)
    print(f"Episode: {episode}, Reward: {tot_reward}, Mean of 100 cons episodes: {mean_100ep}")
    if mean_100ep > env_par['threshold']:
        print(f"Problem is  solved.")
        policy.network.save(agent_path)
        break

    # Save data
    with train_writer.as_default():
        tf.summary.scalar('reward', tot_reward, step=episode)

# Close the environment
CP.env.close()

# Print the summary of the solution
if mean_100ep > env_par['threshold']:
    print(f"\n\nProblem is solved after {episode} Episode with the mean reward {mean_100ep} over the last 100 episodes")


Episode: 0, Reward: 11.0, Mean of 100 cons episodes: 0
Episode: 1, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 2, Reward: 16.0, Mean of 100 cons episodes: 0
Episode: 3, Reward: 8.0, Mean of 100 cons episodes: 0
Episode: 4, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 5, Reward: 9.0, Mean of 100 cons episodes: 0
Episode: 6, Reward: 9.0, Mean of 100 cons episodes: 0
Episode: 7, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 8, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 9, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 10, Reward: 11.0, Mean of 100 cons episodes: 0
Episode: 11, Reward: 8.0, Mean of 100 cons episodes: 0
Episode: 12, Reward: 13.0, Mean of 100 cons episodes: 0
Episode: 13, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 14, Reward: 8.0, Mean of 100 cons episodes: 0
Episode: 15, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 16, Reward: 10.0, Mean of 100 cons episodes: 0
Episode: 17, Reward: 10.0, Mean of 100 cons episodes: 0
Episode

Episode: 144, Reward: 9.0, Mean of 100 cons episodes: 9.78
Episode: 145, Reward: 8.0, Mean of 100 cons episodes: 9.79
Episode: 146, Reward: 9.0, Mean of 100 cons episodes: 9.78
Episode: 147, Reward: 8.0, Mean of 100 cons episodes: 9.76
Episode: 148, Reward: 16.0, Mean of 100 cons episodes: 9.75
Episode: 149, Reward: 9.0, Mean of 100 cons episodes: 9.75
Episode: 150, Reward: 8.0, Mean of 100 cons episodes: 9.79
Episode: 151, Reward: 10.0, Mean of 100 cons episodes: 9.79
Episode: 152, Reward: 10.0, Mean of 100 cons episodes: 9.78
Episode: 153, Reward: 10.0, Mean of 100 cons episodes: 9.8
Episode: 154, Reward: 9.0, Mean of 100 cons episodes: 9.76
Episode: 155, Reward: 10.0, Mean of 100 cons episodes: 9.76
Episode: 156, Reward: 12.0, Mean of 100 cons episodes: 9.77
Episode: 157, Reward: 9.0, Mean of 100 cons episodes: 9.77
Episode: 158, Reward: 10.0, Mean of 100 cons episodes: 9.78
Episode: 159, Reward: 8.0, Mean of 100 cons episodes: 9.76
Episode: 160, Reward: 12.0, Mean of 100 cons episo

Episode: 285, Reward: 8.0, Mean of 100 cons episodes: 9.78
Episode: 286, Reward: 9.0, Mean of 100 cons episodes: 9.78
Episode: 287, Reward: 10.0, Mean of 100 cons episodes: 9.76
Episode: 288, Reward: 8.0, Mean of 100 cons episodes: 9.75
Episode: 289, Reward: 10.0, Mean of 100 cons episodes: 9.75
Episode: 290, Reward: 11.0, Mean of 100 cons episodes: 9.75
Episode: 291, Reward: 10.0, Mean of 100 cons episodes: 9.74
Episode: 292, Reward: 9.0, Mean of 100 cons episodes: 9.76
Episode: 293, Reward: 12.0, Mean of 100 cons episodes: 9.76
Episode: 294, Reward: 9.0, Mean of 100 cons episodes: 9.76
Episode: 295, Reward: 10.0, Mean of 100 cons episodes: 9.77
Episode: 296, Reward: 8.0, Mean of 100 cons episodes: 9.76
Episode: 297, Reward: 10.0, Mean of 100 cons episodes: 9.76
Episode: 298, Reward: 10.0, Mean of 100 cons episodes: 9.74
Episode: 299, Reward: 9.0, Mean of 100 cons episodes: 9.75
Episode: 300, Reward: 11.0, Mean of 100 cons episodes: 9.75
Episode: 301, Reward: 11.0, Mean of 100 cons ep

Episode: 423, Reward: 8.0, Mean of 100 cons episodes: 9.82
Episode: 424, Reward: 10.0, Mean of 100 cons episodes: 9.85
Episode: 425, Reward: 9.0, Mean of 100 cons episodes: 9.83
Episode: 426, Reward: 9.0, Mean of 100 cons episodes: 9.84
Episode: 427, Reward: 12.0, Mean of 100 cons episodes: 9.85
Episode: 428, Reward: 12.0, Mean of 100 cons episodes: 9.85
Episode: 429, Reward: 10.0, Mean of 100 cons episodes: 9.88
Episode: 430, Reward: 12.0, Mean of 100 cons episodes: 9.89
Episode: 431, Reward: 10.0, Mean of 100 cons episodes: 9.89
Episode: 432, Reward: 11.0, Mean of 100 cons episodes: 9.92
Episode: 433, Reward: 10.0, Mean of 100 cons episodes: 9.92
Episode: 434, Reward: 8.0, Mean of 100 cons episodes: 9.94
Episode: 435, Reward: 10.0, Mean of 100 cons episodes: 9.94
Episode: 436, Reward: 10.0, Mean of 100 cons episodes: 9.87
Episode: 437, Reward: 8.0, Mean of 100 cons episodes: 9.88
Episode: 438, Reward: 9.0, Mean of 100 cons episodes: 9.89
Episode: 439, Reward: 9.0, Mean of 100 cons ep

Episode: 559, Reward: 10.0, Mean of 100 cons episodes: 10.89
Episode: 560, Reward: 10.0, Mean of 100 cons episodes: 10.9
Episode: 561, Reward: 10.0, Mean of 100 cons episodes: 10.89
Episode: 562, Reward: 9.0, Mean of 100 cons episodes: 10.87
Episode: 563, Reward: 13.0, Mean of 100 cons episodes: 10.88
Episode: 564, Reward: 11.0, Mean of 100 cons episodes: 10.85
Episode: 565, Reward: 11.0, Mean of 100 cons episodes: 10.88
Episode: 566, Reward: 14.0, Mean of 100 cons episodes: 10.87
Episode: 567, Reward: 17.0, Mean of 100 cons episodes: 10.86
Episode: 568, Reward: 13.0, Mean of 100 cons episodes: 10.88
Episode: 569, Reward: 13.0, Mean of 100 cons episodes: 10.94
Episode: 570, Reward: 14.0, Mean of 100 cons episodes: 10.94
Episode: 571, Reward: 17.0, Mean of 100 cons episodes: 10.96
Episode: 572, Reward: 17.0, Mean of 100 cons episodes: 11.0
Episode: 573, Reward: 19.0, Mean of 100 cons episodes: 11.06
Episode: 574, Reward: 17.0, Mean of 100 cons episodes: 11.12
Episode: 575, Reward: 20.0,

...

Episode: 2282, Reward: 121.0, Mean of 100 cons episodes: 113.13
Episode: 2283, Reward: 115.0, Mean of 100 cons episodes: 113.2
Episode: 2284, Reward: 127.0, Mean of 100 cons episodes: 113.31
Episode: 2285, Reward: 100.0, Mean of 100 cons episodes: 113.46
Episode: 2286, Reward: 117.0, Mean of 100 cons episodes: 113.63
Episode: 2287, Reward: 112.0, Mean of 100 cons episodes: 113.64
Episode: 2288, Reward: 118.0, Mean of 100 cons episodes: 113.83
Episode: 2289, Reward: 114.0, Mean of 100 cons episodes: 113.77
Episode: 2290, Reward: 107.0, Mean of 100 cons episodes: 113.9
Episode: 2291, Reward: 101.0, Mean of 100 cons episodes: 113.92
Episode: 2292, Reward: 110.0, Mean of 100 cons episodes: 113.96
Episode: 2293, Reward: 93.0, Mean of 100 cons episodes: 113.72
Episode: 2294, Reward: 114.0, Mean of 100 cons episodes: 113.74
Episode: 2295, Reward: 93.0, Mean of 100 cons episodes: 113.65
Episode: 2296, Reward: 108.0, Mean of 100 cons episodes: 113.73
Episode: 2297, Reward: 83.0, Mean of 100 con

Episode: 2412, Reward: 123.0, Mean of 100 cons episodes: 113.34
Episode: 2413, Reward: 115.0, Mean of 100 cons episodes: 113.2
Episode: 2414, Reward: 121.0, Mean of 100 cons episodes: 113.61
Episode: 2415, Reward: 95.0, Mean of 100 cons episodes: 113.85
Episode: 2416, Reward: 120.0, Mean of 100 cons episodes: 114.08
Episode: 2417, Reward: 88.0, Mean of 100 cons episodes: 114.1
Episode: 2418, Reward: 137.0, Mean of 100 cons episodes: 114.31
Episode: 2419, Reward: 95.0, Mean of 100 cons episodes: 114.47
Episode: 2420, Reward: 94.0, Mean of 100 cons episodes: 115.14
Episode: 2421, Reward: 100.0, Mean of 100 cons episodes: 115.4
Episode: 2422, Reward: 94.0, Mean of 100 cons episodes: 115.52
Episode: 2423, Reward: 116.0, Mean of 100 cons episodes: 115.65
Episode: 2424, Reward: 110.0, Mean of 100 cons episodes: 115.69
Episode: 2425, Reward: 107.0, Mean of 100 cons episodes: 116.05
Episode: 2426, Reward: 101.0, Mean of 100 cons episodes: 116.37
Episode: 2427, Reward: 112.0, Mean of 100 cons e

Episode: 2542, Reward: 133.0, Mean of 100 cons episodes: 120.13
Episode: 2543, Reward: 111.0, Mean of 100 cons episodes: 119.62
Episode: 2544, Reward: 99.0, Mean of 100 cons episodes: 119.77
Episode: 2545, Reward: 156.0, Mean of 100 cons episodes: 119.71
Episode: 2546, Reward: 141.0, Mean of 100 cons episodes: 119.36
Episode: 2547, Reward: 97.0, Mean of 100 cons episodes: 119.73
Episode: 2548, Reward: 118.0, Mean of 100 cons episodes: 119.77
Episode: 2549, Reward: 107.0, Mean of 100 cons episodes: 119.46
Episode: 2550, Reward: 146.0, Mean of 100 cons episodes: 118.93
Episode: 2551, Reward: 124.0, Mean of 100 cons episodes: 119.01
Episode: 2552, Reward: 114.0, Mean of 100 cons episodes: 119.23
Episode: 2553, Reward: 107.0, Mean of 100 cons episodes: 119.36
Episode: 2554, Reward: 99.0, Mean of 100 cons episodes: 118.97
Episode: 2555, Reward: 106.0, Mean of 100 cons episodes: 118.84
Episode: 2556, Reward: 96.0, Mean of 100 cons episodes: 118.43
Episode: 2557, Reward: 112.0, Mean of 100 co

Episode: 2671, Reward: 147.0, Mean of 100 cons episodes: 134.95
Episode: 2672, Reward: 170.0, Mean of 100 cons episodes: 135.88
Episode: 2673, Reward: 162.0, Mean of 100 cons episodes: 135.97
Episode: 2674, Reward: 162.0, Mean of 100 cons episodes: 136.52
Episode: 2675, Reward: 159.0, Mean of 100 cons episodes: 136.99
Episode: 2676, Reward: 127.0, Mean of 100 cons episodes: 137.19
Episode: 2677, Reward: 146.0, Mean of 100 cons episodes: 137.55
Episode: 2678, Reward: 178.0, Mean of 100 cons episodes: 137.6
Episode: 2679, Reward: 133.0, Mean of 100 cons episodes: 137.42
Episode: 2680, Reward: 126.0, Mean of 100 cons episodes: 137.83
Episode: 2681, Reward: 188.0, Mean of 100 cons episodes: 138.1
Episode: 2682, Reward: 131.0, Mean of 100 cons episodes: 138.18
Episode: 2683, Reward: 162.0, Mean of 100 cons episodes: 139.01
Episode: 2684, Reward: 161.0, Mean of 100 cons episodes: 139.03
Episode: 2685, Reward: 173.0, Mean of 100 cons episodes: 139.53
Episode: 2686, Reward: 200.0, Mean of 100 

Episode: 2800, Reward: 200.0, Mean of 100 cons episodes: 184.02
Episode: 2801, Reward: 180.0, Mean of 100 cons episodes: 184.04
Episode: 2802, Reward: 200.0, Mean of 100 cons episodes: 184.44
Episode: 2803, Reward: 200.0, Mean of 100 cons episodes: 184.24
Episode: 2804, Reward: 200.0, Mean of 100 cons episodes: 184.24
Episode: 2805, Reward: 200.0, Mean of 100 cons episodes: 184.24
Episode: 2806, Reward: 200.0, Mean of 100 cons episodes: 184.24
Episode: 2807, Reward: 200.0, Mean of 100 cons episodes: 184.24
Episode: 2808, Reward: 176.0, Mean of 100 cons episodes: 184.24
Episode: 2809, Reward: 173.0, Mean of 100 cons episodes: 184.29
Episode: 2810, Reward: 168.0, Mean of 100 cons episodes: 184.31
Episode: 2811, Reward: 180.0, Mean of 100 cons episodes: 184.43
Episode: 2812, Reward: 200.0, Mean of 100 cons episodes: 184.58
Episode: 2813, Reward: 170.0, Mean of 100 cons episodes: 184.72
Episode: 2814, Reward: 157.0, Mean of 100 cons episodes: 185.11
Episode: 2815, Reward: 184.0, Mean of 10

## Results
It will get around 5-6 minutes to run the above cell. You will probably get some WARNING\ERROR. Some of these are related to incompatibility between some libraries. Don't panic. If you get the following at the end, the problem is solved successfully.